In [2]:
!pip install catboost

     |████████████████████████████████| 76.6 MB 55 kB/s 


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_validate


pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.display.max_columns = None

In [4]:
raw_data = pd.read_csv('/content/KPI Project Data 0120_1221.csv')
labeled_ml_data = pd.read_csv('labeled_ml.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (39,40,41,42,43,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,103,104,105,106,107,108,109,114,115,116,117,118,119,141,142,143,144,145,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,205,206,207,208,209,210,211,216,217,218,219,220,221,243,244,245,246,247,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,307,308,309,310,311,312,313,318,319,320,321,322,323,345,346,347,348,349,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,409,410,411,412,413,414,415,420,421,42

In [5]:
# Focus only on Social channel
social_data = raw_data[raw_data.channel == 'Social'].drop(columns=['channel', 'size'])
social_data = social_data.iloc[:,:16]  # Data elements

to_drop = ['funnel', 'publisher', 'lob', 'product',	'theme', 'kpi_audience', 'creative_versions',
'price', 'price_placement', 'discount', 'offer_placement', 'video_type', 'length']

# Merge away outliers and concatenate target features
ml_data = pd.merge(social_data, labeled_ml_data.drop(to_drop, axis=1), left_on='Creative ID', right_on='Creative ID')
ml_data.drop(['Creative ID', 'offer_group', 'asset_type'], axis=1, inplace=True)
ml_data

,funnel,publisher,lob,product,theme,kpi_audience,creative_versions,price,price_placement,discount,offer_placement,video_type,length,discount_applied,Mean_Spend,total_spend,total_Impressions,Mean_CTR,Mean_CVRS,Mean_CVRBF,Mean_CVRSV,Peak_CTR,Peak_CVRS,Peak_CVRBF,Peak_CVRSV,Med_CTR,Med_CVRS,Med_CVRBF,Med_CVRSV,CTR_W0,CTR_W1,CTR_W2,CTR_W3,CTR_W4,CTR_W5,CTR_W6,CTR_W7,CTR_W8,CTR_W9,CTR_W10,CTR_W11,CTR_W12,CTR_W13,CTR_W14,CTR_W15,CTR_W16,CTR_W17,CTR_W18,CTR_W19,CTR_W20,CTR_W21,CTR_W22,CTR_W23,CTR_W24,CTR_W25,CTR_W26,CTR_W27,CTR_W28,CTR_W29,CTR_W30,CTR_W31,CTR_W32,CTR_W33,CTR_W34,CTR_W35,CTR_W36,CTR_W37,CTR_W38,CTR_W39,CTR_W40,CVR(BF)_W0,CVR(BF)_W1,CVR(BF)_W2,CVR(BF)_W3,CVR(BF)_W4,CVR(BF)_W5,CVR(BF)_W6,CVR(BF)_W7,CVR(BF)_W8,CVR(BF)_W9,CVR(BF)_W10,CVR(BF)_W11,CVR(BF)_W12,CVR(BF)_W13,CVR(BF)_W14,CVR(BF)_W15,CVR(BF)_W16,CVR(BF)_W17,CVR(BF)_W18,CVR(BF)_W19,CVR(BF)_W20,CVR(BF)_W21,CVR(BF)_W22,CVR(BF)_W23,CVR(BF)_W24,CVR(BF)_W25,CVR(BF)_W26,CVR(BF)_W27,CVR(BF)_W28,CVR(BF)_W29,CVR(BF)_W30,CVR(BF)_W31,CVR(BF)_W32,CVR(BF)_W33,CVR(BF)_W34,CVR(BF)_W35,CVR(BF)_W36,CVR(BF)_W37,CVR(BF)_W38,CVR(BF)_W39,CVR(BF)_W40,CVR(SV)_W0,CVR(SV)_W1,CVR(SV)_W2,CVR(SV)_W3,CVR(SV)_W4,CVR(SV)_W5,CVR(SV)_W6,CVR(SV)_W7,CVR(SV)_W8,CVR(SV)_W9,CVR(SV)_W10,CVR(SV)_W11,CVR(SV)_W12,CVR(SV)_W13,CVR(SV)_W14,CVR(SV)_W15,CVR(SV)_W16,CVR(SV)_W17,CVR(SV)_W18,CVR(SV)_W19,CVR(SV)_W20,CVR(SV)_W21,CVR(SV)_W22,CVR(SV)_W23,CVR(SV)_W24,CVR(SV)_W25,CVR(SV)_W26,CVR(SV)_W27,CVR(SV)_W28,CVR(SV)_W29,CVR(SV)_W30,CVR(SV)_W31,CVR(SV)_W32,CVR(SV)_W33,CVR(SV)_W34,CVR(SV)_W35,CVR(SV)_W36,CVR(SV)_W37,CVR(SV)_W38,CVR(SV)_W39,CVR(SV)_W40,Internet_CVR(S)_W0,Internet_CVR(S)_W1,Internet_CVR(S)_W2,Internet_CVR(S)_W3,Internet_CVR(S)_W4,Internet_CVR(S)_W5,Internet_CVR(S)_W6,Internet_CVR(S)_W7,Internet_CVR(S)_W8,Internet_CVR(S)_W9,Internet_CVR(S)_W10,Internet_CVR(S)_W11,Internet_CVR(S)_W12,Internet_CVR(S)_W13,Internet_CVR(S)_W14,Internet_CVR(S)_W15,Internet_CVR(S)_W16,Internet_CVR(S)_W17,Internet_CVR(S)_W18,Internet_CVR(S)_W19,Internet_CVR(S)_W20,Internet_CVR(S)_W21,Internet_CVR(S)_W22,Internet_CVR(S)_W23,Internet_CVR(S)_W24,Internet_CVR(S)_W25,Internet_CVR(S)_W26,Internet_CVR(S)_W27,Internet_CVR(S)_W28,Internet_CVR(S)_W29,Internet_CVR(S)_W30,Internet_CVR(S)_W31,Internet_CVR(S)_W32,Internet_CVR(S)_W33,Internet_CVR(S)_W34,Internet_CVR(S)_W35,Internet_CVR(S)_W36,Internet_CVR(S)_W37,Internet_CVR(S)_W38,Internet_CVR(S)_W39,Internet_CVR(S)_W40
0,MF,Publisher 1,LOB 1,Product 1,Theme 1,Audience 1,Creative 1,No Price,NaN,0.00,NaN,Stock,:15,0,318.77,4144.00,204729.00,0.54,0.00,0.00,0.00,1.71,0.00,0.00,0.00,1.71,0.00,0.00,0.00,0.98,1.58,0.00,0.00,0.13,0.43,0.48,0.43,0.41,0.90,1.71,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,MF,Publisher 3,LOB 1,Product 1,Theme 1,Audience 2,Creative 1,No Price,NaN,0.00,NaN,Stock,:15,0,2953.33,26580.00,959822.00,0.29,0.25,25.07,0.12,0.49,1.49,37.77,0.27,0.49,1.49,37.77,0.27,0.49,0.35,0.26,0.27,0.27,0.26,0.23,0.24,0.23,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,37.77,35.93,34.39,17.91,25.14,21.20,16.44,25.02,11.82,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0

In [6]:
ml_data.iloc[:,:14].isnull().sum()

funnel                 0
publisher              0
lob                    0
product                0
theme                  0
kpi_audience           0
creative_versions      0
price                  0
price_placement      590
discount               0
offer_placement      590
video_type            49
length                 0
discount_applied       0
dtype: int64

In [11]:
ml_data.loc[ml_data.price_placement.isnull(), 'price_placement'] = 'Missing'
ml_data.loc[ml_data.offer_placement.isnull(), 'offer_placement'] = 'Missing'
ml_data.loc[ml_data.video_type.isnull(), 'video_type'] = 'Missing'

ml_data.iloc[:,:14].isnull().sum()

funnel               0
publisher            0
lob                  0
product              0
theme                0
kpi_audience         0
creative_versions    0
price                0
price_placement      0
discount             0
offer_placement      0
video_type           0
length               0
discount_applied     0
dtype: int64

In [8]:
# Funnel specific
ml_data_UF = ml_data[ml_data.funnel == 'UF'].drop('funnel', axis=1)
ml_data_MF = ml_data[ml_data.funnel == 'MF'].drop('funnel', axis=1)
ml_data_LF = ml_data[ml_data.funnel == 'LF'].drop('funnel', axis=1)

In [20]:
# CTR
ctrs = [col for col in ml_data.columns if col.startswith('CTR')][:8]  # First 8 weeks

X = ml_data_UF[['publisher', 'kpi_audience', 'creative_versions', 'discount', 'lob', 'video_type']]
y = ml_data_UF[ctrs]

# Split train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Columns to encode
cat_cols = ['publisher', 'kpi_audience', 'creative_versions', 'lob', 'video_type']

# Preprocessing
#one_hot = OneHotEncoder()

# ct = make_column_transformer(
#     (one_hot, one_hot_cols),
#     remainder='passthrough'
# )

# Pipeline
#rf = RandomForestRegressor(random_state=123)
#pipe = make_pipeline(ct, rf)
for ctr in ctrs:
  cb = CatBoostRegressor(iterations=50, random_state=123)
  cb.fit(X_train, y_train[ctr], cat_features=cat_cols)
  print(f'{ctr}: {cb.score(X_test, y_test[ctr])}\n\n')

# Model evaluation
#scorers = ['r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error']

#for ctr in ctrs:
#cv = cross_validate(cb, X_train, y_train[ctr], cv=4, scoring=scorers, error_score='raise')
#r2 = cv.get('test_r2').mean().round(2)
#rmse = -1*cv.get('test_neg_root_mean_squared_error').mean().round(4)
#mae = -1*cv.get('test_neg_mean_absolute_error').mean().round(4)
#print(ctr)
#print(f'R2: {r2}\nRMSE: {rmse}\nMAE: {mae}\n')

Learning rate set to 0.43001
0:	learn: 0.1006392	total: 996us	remaining: 48.8ms
1:	learn: 0.0877877	total: 2ms	remaining: 48ms
2:	learn: 0.0803683	total: 3.16ms	remaining: 49.5ms
3:	learn: 0.0757834	total: 4.3ms	remaining: 49.4ms
4:	learn: 0.0731040	total: 5.35ms	remaining: 48.1ms
5:	learn: 0.0682517	total: 6.42ms	remaining: 47.1ms
6:	learn: 0.0657455	total: 7.51ms	remaining: 46.1ms
7:	learn: 0.0650324	total: 8.64ms	remaining: 45.4ms
8:	learn: 0.0639859	total: 9.73ms	remaining: 44.3ms
9:	learn: 0.0619480	total: 10.8ms	remaining: 43.2ms
10:	learn: 0.0616684	total: 12ms	remaining: 42.4ms
11:	learn: 0.0602931	total: 13.1ms	remaining: 41.5ms
12:	learn: 0.0602682	total: 14.1ms	remaining: 40.2ms
13:	learn: 0.0602576	total: 14.7ms	remaining: 37.8ms
14:	learn: 0.0589532	total: 15.8ms	remaining: 36.8ms
15:	learn: 0.0588399	total: 16.8ms	remaining: 35.8ms
16:	learn: 0.0586471	total: 17.9ms	remaining: 34.7ms
17:	learn: 0.0580796	total: 19ms	remaining: 33.7ms
18:	learn: 0.0575587	total: 20.1ms	rem

In [12]:
X.head()

,funnel,publisher,lob,product,theme,kpi_audience,creative_versions,price,price_placement,discount,offer_placement,video_type,length,discount_applied,price_applied
0,MF,Publisher 1,LOB 1,Product 1,Theme 1,Audience 1,Creative 1,No Price,Missing,0.00,Missing,Stock,:15,0,0
1,MF,Publisher 3,LOB 1,Product 1,Theme 1,Audience 2,Creative 1,No Price,Missing,0.00,Missing,Stock,:15,0,0
2,MF,Publisher 4,LOB 1,Product 1,Theme 1,Audience 2,Creative 1,No Price,Missing,0.00,Missing,Stock,:15,0,0
3,MF,Publisher 5,LOB 1,Product 1,Theme 1,Audience 2,Creative 1,No Price,Missing,0.00,Missing,Stock,:15,0,0
4,LF,Publisher 5,LOB 1,Product 2,Theme 1,Audience 2,Creative 2,Price 1,Missing,0.00,Missing,Stock,:15,0,1


In [13]:
y.head()

,Mean_Spend,total_spend,total_Impressions,Mean_CTR,Mean_CVRS,Mean_CVRBF,Mean_CVRSV,Peak_CTR,Peak_CVRS,Peak_CVRBF,Peak_CVRSV,Med_CTR,Med_CVRS,Med_CVRBF,Med_CVRSV,CTR_W0,CTR_W1,CTR_W2,CTR_W3,CTR_W4,CTR_W5,CTR_W6,CTR_W7,CTR_W8,CTR_W9,CTR_W10,CTR_W11,CTR_W12,CTR_W13,CTR_W14,CTR_W15,CTR_W16,CTR_W17,CTR_W18,CTR_W19,CTR_W20,CTR_W21,CTR_W22,CTR_W23,CTR_W24,CTR_W25,CTR_W26,CTR_W27,CTR_W28,CTR_W29,CTR_W30,CTR_W31,CTR_W32,CTR_W33,CTR_W34,CTR_W35,CTR_W36,CTR_W37,CTR_W38,CTR_W39,CTR_W40,CVR(BF)_W0,CVR(BF)_W1,CVR(BF)_W2,CVR(BF)_W3,CVR(BF)_W4,CVR(BF)_W5,CVR(BF)_W6,CVR(BF)_W7,CVR(BF)_W8,CVR(BF)_W9,CVR(BF)_W10,CVR(BF)_W11,CVR(BF)_W12,CVR(BF)_W13,CVR(BF)_W14,CVR(BF)_W15,CVR(BF)_W16,CVR(BF)_W17,CVR(BF)_W18,CVR(BF)_W19,CVR(BF)_W20,CVR(BF)_W21,CVR(BF)_W22,CVR(BF)_W23,CVR(BF)_W24,CVR(BF)_W25,CVR(BF)_W26,CVR(BF)_W27,CVR(BF)_W28,CVR(BF)_W29,CVR(BF)_W30,CVR(BF)_W31,CVR(BF)_W32,CVR(BF)_W33,CVR(BF)_W34,CVR(BF)_W35,CVR(BF)_W36,CVR(BF)_W37,CVR(BF)_W38,CVR(BF)_W39,CVR(BF)_W40,CVR(SV)_W0,CVR(SV)_W1,CVR(SV)_W2,CVR(SV)_W3,CVR(SV)_W4,CVR(SV)_W5,CVR(SV)_W6,CVR(SV)_W7,CVR(SV)_W8,CVR(SV)_W9,CVR(SV)_W10,CVR(SV)_W11,CVR(SV)_W12,CVR(SV)_W13,CVR(SV)_W14,CVR(SV)_W15,CVR(SV)_W16,CVR(SV)_W17,CVR(SV)_W18,CVR(SV)_W19,CVR(SV)_W20,CVR(SV)_W21,CVR(SV)_W22,CVR(SV)_W23,CVR(SV)_W24,CVR(SV)_W25,CVR(SV)_W26,CVR(SV)_W27,CVR(SV)_W28,CVR(SV)_W29,CVR(SV)_W30,CVR(SV)_W31,CVR(SV)_W32,CVR(SV)_W33,CVR(SV)_W34,CVR(SV)_W35,CVR(SV)_W36,CVR(SV)_W37,CVR(SV)_W38,CVR(SV)_W39,CVR(SV)_W40,Internet_CVR(S)_W0,Internet_CVR(S)_W1,Internet_CVR(S)_W2,Internet_CVR(S)_W3,Internet_CVR(S)_W4,Internet_CVR(S)_W5,Internet_CVR(S)_W6,Internet_CVR(S)_W7,Internet_CVR(S)_W8,Internet_CVR(S)_W9,Internet_CVR(S)_W10,Internet_CVR(S)_W11,Internet_CVR(S)_W12,Internet_CVR(S)_W13,Internet_CVR(S)_W14,Internet_CVR(S)_W15,Internet_CVR(S)_W16,Internet_CVR(S)_W17,Internet_CVR(S)_W18,Internet_CVR(S)_W19,Internet_CVR(S)_W20,Internet_CVR(S)_W21,Internet_CVR(S)_W22,Internet_CVR(S)_W23,Internet_CVR(S)_W24,Internet_CVR(S)_W25,Internet_CVR(S)_W26,Internet_CVR(S)_W27,Internet_CVR(S)_W28,Internet_CVR(S)_W29,Internet_CVR(S)_W30,Internet_CVR(S)_W31,Internet_CVR(S)_W32,Internet_CVR(S)_W33,Internet_CVR(S)_W34,Internet_CVR(S)_W35,Internet_CVR(S)_W36,Internet_CVR(S)_W37,Internet_CVR(S)_W38,Internet_CVR(S)_W39,Internet_CVR(S)_W40
0,318.77,4144.00,204729.00,0.54,0.00,0.00,0.00,1.71,0.00,0.00,0.00,1.71,0.00,0.00,0.00,0.98,1.58,0.00,0.00,0.13,0.43,0.48,0.43,0.41,0.90,1.71,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2953.33,26580.00,959822.00,0.29,0.25,25.07,0.12,0.49,1.49,37.77,0.27,0.49,1.49,37.77,0.27,0.49,0.35,0.26,0.27,0.27,0.26,0.23,0.24,0.23,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,37.77,35.93,34.39,17.91,25.14,21.20,16.44,25.02,11.82,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.27,0.15,0.11,0.12,0.11,0.09,0.08,0.10,0.09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.49,0.17,0.17,0.16,0.15,0.15,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00

In [ ]:
# Predicting only first 8 weeks
ctr = [f'CTR_W{i}' for i in range(9)]
cvrsv = [f'CVR(SV){i}' for i in range(9)]
cvrbf = [f'CVR(BF){i}' for i in range(9)]
cvrs = [f'CVR(S){i}' for i in range(9)]